In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report

import re

import joblib

import string

In [2]:
fake =pd.read_csv('Fake.csv')
true =pd.read_csv('True.csv')

In [3]:
fake.head()


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
true.head()


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
fake['class']=0
true['class']=1

In [6]:
data =pd.concat([fake,true],axis=0)

In [9]:
data.sample(10)

,title,text,subject,date,class
10350,DNC CHAIR Pulls Tired Race Card: Claims “Racis...,DNC Chair Tom Perez is a fear mongering racist...,politics,"Jul 20, 2017",0
4142,WATCH: President Obama Tells Whiny Trump To S...,Most Americans wish that the 2016 Presidential...,News,"October 18, 2016",0
8520,"Bentley, former Congresswoman from Maryland, d...","(Reuters) - Helen Delich Bentley, a former jou...",politicsNews,"August 7, 2016",1
3501,Major City SLAPS Greedy CEOs Like Trump With ...,This is HUGE news!The city council of Portland...,News,"December 8, 2016",0
5664,Trump Makes ANOTHER Racist Joke About Mexican...,Donald Trump has been making racist comments a...,News,"June 30, 2016",0
13527,"EU's Tusk: Africa, EU must cooperate to end 'h...",ABIDJAN (Reuters) - Europe and Africa have joi...,worldnews,"November 29, 2017",1
15903,"Khamenei says Iran, Russia should cooperate to...",TEHRAN (Reuters) - Iranian Supreme Leader Ayat...,worldnews,"November 1, 2017",1
9700,THE CHIEF IS HERE! JOHN KELLY Surprises White ...,This is great! Gen. John Kelly surprised the W...,politics,"Oct 12, 2017",0
18808,Brexit bill gives UK ministers 'excessively wi...,LONDON (Reuters) - A parliamentary committee o...,worldnews,"September 28, 2017",1
3629,Factbox: Key points of Trump's 2018 budget pro...,(Reuters) - The following are stories and tabl...,politicsNews,"May 23, 2017",1


In [10]:
data = data.drop(["title","subject","date"],axis=1)

In [12]:
data.reset_index(inplace=True)

In [15]:
data.drop(['index'],axis=1,inplace=True)

In [18]:
data.sample(10)

,text,class
43587,ANKARA (Reuters) - Turkey said on Thursday it ...,1
4735,"The city of Portland, Maine has been thrown in...",0
35542,SANTIAGO (Reuters) - Center-left presidential ...,1
31005,WASHINGTON (Reuters) - Democrat Hillary Clinto...,1
29889,WASHINGTON (Reuters) - The NATO alliance’s mut...,1
2111,A sit-down meeting with German Chancellor Ange...,0
20111,Does anyone else get the sense Hillary is not ...,0
10592,"After allegedly registering 10,000 Muslims to ...",0
8345,"The Iowa caucuses have come and gone, leaving ...",0
30622,LIMA (Reuters) - U.S. President Barack Obama d...,1


In [20]:


def clean_text(text):
    text = text.lower()  # Convert text to lowercase
    
    text = re.sub(r"\[.*?\]", "", text)  # Remove content within brackets
    text = re.sub(r"\W", " ", text)  # Replace non-word characters with space
    text = re.sub(r"https?://\S+|www\.\S+", "", text)  # Remove URLs
    text = re.sub(r"<.*?>", "", text)  # Remove HTML tags
    text = re.sub("[%s]" % re.escape(string.punctuation), "", text)  # Remove punctuation
    text = re.sub(r"\n", " ", text)  # Remove newlines
    text = re.sub(r"\w*\d\w*", "", text)  # Remove words containing numbers

    return text

In [21]:
data["text"]= data["text"].apply(clean_text)

In [22]:
x=data["text"]
y=data["class"]

xtrain, xtest, ytrain, ytest=train_test_split(x,y,test_size=0.25,random_state=42)

In [25]:
vectorizer=TfidfVectorizer()
xv_train=vectorizer.fit_transform(xtrain)
xv_test=vectorizer.transform(xtest)

In [26]:
lr=LogisticRegression()
lr.fit(xv_train,ytrain)

LogisticRegression()

In [29]:
prediction=lr.predict(xv_test)
lr.score(xv_test,ytest)

0.9858351893095768

In [30]:
print(classification_report(ytest,prediction))


              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [33]:
joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(lr,"lr_model.jb")

['lr_model.jb']